In [1]:
import numpy as np

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import pandas as pd
from us_state_abbrev import abbrev_to_us_state as un_abbrev
from flask import Flask, jsonify, render_template


In [2]:
#################################################
# Database Setup 
#################################################

engine = create_engine("sqlite:///ufo.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)




In [3]:
# Save references to the tables
geo = Base.classes.state_geo
Ufos = Base.classes.ufo_sightings

In [ ]:
#app = Flask(__name__)

In [ ]:
@app.route("/api/v1.0/map_info")
def map_info():
     # Open session
    session = Session(engine)

    #queries
    state_info=session.query(geo.state,geo.population,geo.geometry).all()
    ufo_info=session.query(Ufos.state).count()



In [4]:
session = Session(engine)
ufo_info=session.query(Ufos.state).count()


56984

In [32]:
session = Session(engine)
state_info=session.query(geo.state,geo.population,geo.geometry).all()
info=list(np.ravel(state_info[0]))
sightings=session.query(Ufos).filter(Ufos.state==info[0]).count()
feature_dict={"properties":{}}
properties=feature_dict["properties"]
feature_dict["id"]=1
properties["name"]=info[0]
properties["population"]=int(info[1])
properties["sightings"]=sightings
properties["per_capita"]=sightings/int(info[1])
properties["geometry"]=eval(info[2])
feature_dict

{'properties': {'name': 'AL',
  'population': 5039877,
  'sightings': 1350,
  'per_capita': 0.00026786368000647633,
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-87.359296, 35.00118],
     [-85.606675, 34.984749],
     [-85.431413, 34.124869],
     [-85.184951, 32.859696],
     [-85.069935, 32.580372],
     [-84.960397, 32.421541],
     [-85.004212, 32.322956],
     [-84.889196, 32.262709],
     [-85.058981, 32.13674],
     [-85.053504, 32.01077],
     [-85.141136, 31.840985],
     [-85.042551, 31.539753],
     [-85.113751, 31.27686],
     [-85.004212, 31.003013],
     [-85.497137, 30.997536],
     [-87.600282, 30.997536],
     [-87.633143, 30.86609],
     [-87.408589, 30.674397],
     [-87.446927, 30.510088],
     [-87.37025, 30.427934],
     [-87.518128, 30.280057],
     [-87.655051, 30.247195],
     [-87.90699, 30.411504],
     [-87.934375, 30.657966],
     [-88.011052, 30.685351],
     [-88.10416, 30.499135],
     [-88.137022, 30.318396],
     [-88.394438, 30.367688],
   

In [33]:
session = Session(engine)
state_info=session.query(geo.state,geo.population,geo.geometry).all()
geoJson={"type":"FeatureCollection",
            "features":[]}
for i in range(len(state_info)):
    info=list(np.ravel(state_info[i]))
    sightings=session.query(Ufos).filter(Ufos.state==info[0]).count()
    feature_dict={"properties":{}}
    properties=feature_dict["properties"]
    feature_dict["id"]=i+1
    properties["name"]=info[0]
    properties["proper_name"]=un_abbrev[info[0]]
    properties["population"]=info[1]
    properties["sightings"]=session.query(Ufos).filter(Ufos.state==info[0]).count()
    properties["per_capita"]=sightings/int(info[1])
    properties["geometry"]=eval(info[2])
    geoJson["features"].append(feature_dict)

In [22]:
state_info=session.query(geo.state,geo.population,geo.geometry).all()
geoJson={"type":"FeatureCollection",
             "features":[]}
#loop through states
for i in range(len(state_info)):
    #unravel tuples
    info=list(np.ravel(state_info[i]))
    if info[0] != "PR":
        #count sightings in the state of this paricular row
        sightings=session.query(Ufos).filter(Ufos.state==info[0]).count()
        #(almost) blank dictionary for the feature
        feature_dict={"type":"Feature",
                        "properties":{}}
        feature_dict["id"]=i+1
        properties=feature_dict["properties"]
        properties["name"]=info[0]
        properties["proper_name"]=un_abbrev[info[0]]
        properties["population"]=info[1]
        properties["sightings"]=session.query(Ufos).filter(Ufos.state==info[0]).count()
        properties["per_capita"]=(sightings/int(info[1]))*100000
        properties["geometry"]=eval(info[2])
        geoJson["features"].append(feature_dict)

In [23]:
sight_list=[]
percap_list=[]
name_list=[]
for feature in geoJson["features"]:
    properties=feature["properties"]
    sight=properties["sightings"]
    per_cap=properties["per_capita"]
    _name=properties["name"]
    sight_list.append(sight)
    percap_list.append(per_cap)
    name_list.append(_name)


In [24]:
map_data_df=pd.DataFrame({"state":name_list,"sightings":sight_list,"per_capita":percap_list})
map_data_df

,state,sightings,per_capita
0,AL,675,13.393184
1,AK,291,39.717582
2,AZ,2141,29.424231
3,AR,541,17.879031
4,CA,5996,15.281169
5,CO,1622,27.907446
6,CT,1013,28.095209
7,DE,209,20.829513
8,DC,38,5.671219
9,FL,4096,18.805270


In [28]:
map_data_df['sights_bin'] = pd.qcut(map_data_df['sightings'], q=8)
map_data_df['percap_bin'] = pd.qcut(map_data_df['per_capita'], q=8)
map_data_df['sights_bin'].value_counts().sort_index()

(37.999, 292.75]     7
(292.75, 470.0]      6
(470.0, 649.0]       7
(649.0, 809.0]       6
(809.0, 1128.75]     6
(1128.75, 1372.5]    6
(1372.5, 2057.0]     6
(2057.0, 5996.0]     7
Name: sights_bin, dtype: int64

In [29]:
map_data_df['percap_bin'].value_counts().sort_index()

(5.67, 12.24]       7
(12.24, 15.04]      6
(15.04, 16.353]     6
(16.353, 17.879]    7
(17.879, 21.133]    6
(21.133, 27.553]    6
(27.553, 38.26]     6
(38.26, 49.569]     7
Name: percap_bin, dtype: int64